In [1]:
import torch as th
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch import nn, optim

In [2]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,),(0.5,))])

In [3]:
mnist_trainset = datasets.MNIST(root="./data",train=True, download=True, transform = transform)
trainloader = th.utils.data.DataLoader(mnist_trainset, batch_size=64, shuffle=True)

100.1%Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw
113.5%Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw
100.4%Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw
180.4%Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw
Processing...
Done!


In [5]:
testloader = th.utils.data.DataLoader(
    datasets.MNIST('./data',train=False, transform=transforms.Compose([
        transforms .ToTensor(),
        transforms.Normalize((0.5,),(0.5,))
    ])),
    batch_size=64, shuffle=True
)

In [7]:
trainloader.dataset

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5,), std=(0.5,))
           )

In [9]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 1000
        self.epochs = 5
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 30
        self.save_model = False

args = Arguments()

In [12]:
trainloader.dataset.data[0].shape

torch.Size([28, 28])

In [14]:
def trainSingle(args, model, train_loader, epoch):
    
    criterion = nn.NLLLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.003)

    for batch_idx, (data, target) in enumerate(train_loader):
        
        optimizer.zero_grad()
        output = model(data.view(data.shape[0], -1))
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % args.log_interval == 0:

            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(train_loader)*args.batch_size,
                100. * batch_idx / len(train_loader), loss.item()
            ))

In [15]:
def test(args, model, test_loader):
    
    model.eval()
    test_loss = 0
    correct = 0
    criterion = nn.NLLLoss()

    with th.no_grad():
        for data, target in test_loader:
            output = model(data.view(data.shape[0], -1))
            test_loss += criterion(output, target).item()
            pred = output.argmax(1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average Loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct/len(test_loader.dataset)
    ))

In [16]:
model = nn.Sequential(
    nn.Linear(784,128),
    nn.ReLU(),
    nn.Linear(128,64),
    nn.ReLU(),
    nn.Linear(64,10),
    nn.LogSoftmax(dim=1)
)

In [23]:
def runSingle(args_, model_, train_loader, test_loader):

    for epoch in range(1, args_.epochs+1):
        trainSingle(args_, model_, train_loader, epoch)
        test(args_, model_, test_loader)

In [24]:
runSingle(args, model, trainloader, testloader)

Train Epoch: 1 [0/60032 (0%)]	Loss: 2.306031
Train Epoch: 1 [1920/60032 (3%)]	Loss: 2.280874
Train Epoch: 1 [3840/60032 (6%)]	Loss: 2.274052
Train Epoch: 1 [5760/60032 (10%)]	Loss: 2.264879
Train Epoch: 1 [7680/60032 (13%)]	Loss: 2.240621
Train Epoch: 1 [9600/60032 (16%)]	Loss: 2.221420
Train Epoch: 1 [11520/60032 (19%)]	Loss: 2.211297
Train Epoch: 1 [13440/60032 (22%)]	Loss: 2.163599
Train Epoch: 1 [15360/60032 (26%)]	Loss: 2.132312
Train Epoch: 1 [17280/60032 (29%)]	Loss: 2.106156
Train Epoch: 1 [19200/60032 (32%)]	Loss: 2.092826
Train Epoch: 1 [21120/60032 (35%)]	Loss: 2.065591
Train Epoch: 1 [23040/60032 (38%)]	Loss: 2.063183
Train Epoch: 1 [24960/60032 (42%)]	Loss: 1.968164
Train Epoch: 1 [26880/60032 (45%)]	Loss: 1.965822
Train Epoch: 1 [28800/60032 (48%)]	Loss: 1.942527
Train Epoch: 1 [30720/60032 (51%)]	Loss: 1.929292
Train Epoch: 1 [32640/60032 (54%)]	Loss: 1.907287
Train Epoch: 1 [34560/60032 (58%)]	Loss: 1.704797
Train Epoch: 1 [36480/60032 (61%)]	Loss: 1.786498
Train Epoch:

In [25]:
# for federated learning we need to create federatedlaoder
# but first need to define machines
import syft as sy

In [26]:
hook = sy.TorchHook(th)

bob = sy.VirtualWorker(hook, id='bob')
alice = sy.VirtualWorker(hook, id='alice')

In [27]:
federated_train_loader = sy.FederatedDataLoader( # <-- this is now a FederatedDataLoader 
    datasets.MNIST('./data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.5,), (0.5,))
                   ]))
    .federate((bob, alice)), # <-- NEW: we distribute the dataset across all the workers, it's now a FederatedDataset
    batch_size=64, shuffle=True)

In [28]:
def trainFL(args, model, federated_train_loader, epoch):
    
    criterion = nn.NLLLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.003)


    for batch_idx, (data, target) in enumerate(federated_train_loader): # <-- now it is a distributed dataset
        model.send(data.location) # <-- NEW: send the model to the right location        
        #data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data.view(data.shape[0], -1))
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        model.get() # <-- NEW: get the model back
        
        if batch_idx % args.log_interval == 0:
            loss = loss.get() # <-- NEW: get the loss back
            print('Location: -->{}'.format(data.location))
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(federated_train_loader) * args.batch_size,
                100. * batch_idx / len(federated_train_loader), loss.item()
                ))

In [29]:
def runFL(args_, model_, federated_train_loader, test_loader):

    for epoch in range(1, args_.epochs+1):
        trainFL(args_, model_, federated_train_loader, epoch)
        test(args_, model_, test_loader)

In [30]:
runFL(args, model, federated_train_loader, testloader)

Location: --><VirtualWorker id:bob #objects:5>
Train Epoch: 1 [0/60032 (0%)]	Loss: 0.363805
Location: --><VirtualWorker id:bob #objects:5>
Train Epoch: 1 [1920/60032 (3%)]	Loss: 0.376995
Location: --><VirtualWorker id:bob #objects:5>
Train Epoch: 1 [3840/60032 (6%)]	Loss: 0.451310
Location: --><VirtualWorker id:bob #objects:5>
Train Epoch: 1 [5760/60032 (10%)]	Loss: 0.279799
Location: --><VirtualWorker id:bob #objects:5>
Train Epoch: 1 [7680/60032 (13%)]	Loss: 0.537798
Location: --><VirtualWorker id:bob #objects:5>
Train Epoch: 1 [9600/60032 (16%)]	Loss: 0.318719
Location: --><VirtualWorker id:bob #objects:5>
Train Epoch: 1 [11520/60032 (19%)]	Loss: 0.240266
Location: --><VirtualWorker id:bob #objects:5>
Train Epoch: 1 [13440/60032 (22%)]	Loss: 0.379196
Location: --><VirtualWorker id:bob #objects:5>
Train Epoch: 1 [15360/60032 (26%)]	Loss: 0.345611
Location: --><VirtualWorker id:bob #objects:5>
Train Epoch: 1 [17280/60032 (29%)]	Loss: 0.261623
Location: --><VirtualWorker id:bob #object